# データサイエンス100本ノック（構造化データ加工編） - Julia

## はじめに
- 初めに以下のセルを実行してください
- ~~必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います~~
- ~~pandas等、~~利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください~~（"!pip install ライブラリ名"でインストールも可能）~~
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
using Dates
using DataFrames
using DataFramesMeta
using StatsBase
using ShiftedArrays: lag
using MLJBase: partition
using MLUtils: undersample, getobs
using PyCall
import CSV

df_customer = CSV.read("data/src/customer.csv", DataFrame, types=Dict(r"_id$" => String, r"_cd$" => String, :application_date => String))
df_category = CSV.read("data/src/category.csv", DataFrame, types=Dict(r"_cd$" => String))
df_product = CSV.read("data/src/product.csv", DataFrame, types=Dict(r"_cd$" => String))
df_receipt = CSV.read("data/src/receipt.csv", DataFrame, types=Dict(r"_id$" => String, r"_cd$" => String))
df_store = CSV.read("data/src/store.csv", DataFrame, types=Dict(r"_cd$" => String, :tel_no => String))
df_geocode = CSV.read("data/src/geocode.csv", DataFrame, types=Dict(r"_cd$" => String, :street => String));

In [2]:
const relativedelta = pyimport("dateutil.relativedelta").relativedelta;

# 演習問題

---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [3]:
@time \
first(df_receipt, 10)

  0.000001 seconds


Row,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
,Int64,Int64,String,Int64,Int64,String,String,Int64,Int64
1,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
2,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
3,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
4,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
5,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
6,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
7,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
8,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
9,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770


---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [4]:
@time \
df_receipt[!, [:sales_ymd, :customer_id, :product_cd, :amount]] |>
    df -> first(df, 10)

  0.000001 seconds


Row,sales_ymd,customer_id,product_cd,amount
,Int64,String,String,Int64
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770


In [5]:
@time \
@chain df_receipt begin
    @select :sales_ymd :customer_id :product_cd :amount
    first(10)
end

  0.000006 seconds


Row,sales_ymd,customer_id,product_cd,amount
,Int64,String,String,Int64
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770


---
> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更しながら抽出すること。

In [6]:
@time \
@chain df_receipt begin
    @select $(:sales_ymd => :sales_date) :customer_id :product_cd :amount
    first(10)
end

  0.000005 seconds


Row,sales_date,customer_id,product_cd,amount
,Int64,String,String,Int64
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770


---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [7]:
# DataFramesの関数で処理
@time \
@chain df_receipt begin
    select(:sales_ymd, :customer_id, :product_cd, :amount)
    subset(:customer_id => ByRow(==("CS018205000001")))
end

  0.000001 seconds


Row,sales_ymd,customer_id,product_cd,amount
,Int64,String,String,Int64
1,20180911,CS018205000001,P071401012,2200
2,20180414,CS018205000001,P060104007,600
3,20170614,CS018205000001,P050206001,990
4,20170614,CS018205000001,P060702015,108
5,20190216,CS018205000001,P071005024,102
6,20180414,CS018205000001,P071101002,278
7,20190226,CS018205000001,P070902035,168
8,20190924,CS018205000001,P060805001,495
9,20190226,CS018205000001,P071401020,2200


In [8]:
# DataFramesMetaのマクロを使用
@time \
@chain df_receipt begin
    @select :sales_ymd :customer_id :product_cd :amount
    @rsubset :customer_id == "CS018205000001"
end

  0.000002 seconds


Row,sales_ymd,customer_id,product_cd,amount
,Int64,String,String,Int64
1,20180911,CS018205000001,P071401012,2200
2,20180414,CS018205000001,P060104007,600
3,20170614,CS018205000001,P050206001,990
4,20170614,CS018205000001,P060702015,108
5,20190216,CS018205000001,P071005024,102
6,20180414,CS018205000001,P071101002,278
7,20190226,CS018205000001,P070902035,168
8,20190924,CS018205000001,P060805001,495
9,20190226,CS018205000001,P071401020,2200


---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [9]:
@time \
@chain df_receipt begin
    @select :sales_ymd :customer_id :product_cd :amount
    @rsubset begin
        :customer_id == "CS018205000001"
        :amount >= 1000
    end
end

  0.000001 seconds


Row,sales_ymd,customer_id,product_cd,amount
,Int64,String,String,Int64
1,20180911,CS018205000001,P071401012,2200
2,20190226,CS018205000001,P071401020,2200
3,20180911,CS018205000001,P071401005,1100


---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [10]:
@time \
@chain df_receipt begin
    @select :sales_ymd :customer_id :product_cd :quantity :amount
    @rsubset begin
        :customer_id == "CS018205000001"
        :amount >= 1000 || :quantity >= 5
    end
end

  0.000006 seconds


Row,sales_ymd,customer_id,product_cd,quantity,amount
,Int64,String,String,Int64,Int64
1,20180911,CS018205000001,P071401012,1,2200
2,20180414,CS018205000001,P060104007,6,600
3,20170614,CS018205000001,P050206001,5,990
4,20190226,CS018205000001,P071401020,1,2200
5,20180911,CS018205000001,P071401005,1,1100


---
> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [11]:
@time \
@chain df_receipt begin
    @select :sales_ymd :customer_id :product_cd :amount
    @rsubset begin
        :customer_id == "CS018205000001"
        1000 <= :amount <= 2000
    end
end

  0.000001 seconds


Row,sales_ymd,customer_id,product_cd,amount
,Int64,String,String,Int64
1,20180911,CS018205000001,P071401005,1100


---
> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [12]:
@time \
@chain df_receipt begin
    @select :sales_ymd :customer_id :product_cd :amount
    @rsubset begin
        :customer_id == "CS018205000001"
        :product_cd != "P071401019"
    end
end

  0.000001 seconds


Row,sales_ymd,customer_id,product_cd,amount
,Int64,String,String,Int64
1,20180911,CS018205000001,P071401012,2200
2,20180414,CS018205000001,P060104007,600
3,20170614,CS018205000001,P050206001,990
4,20170614,CS018205000001,P060702015,108
5,20190216,CS018205000001,P071005024,102
6,20180414,CS018205000001,P071101002,278
7,20190226,CS018205000001,P070902035,168
8,20190924,CS018205000001,P060805001,495
9,20190226,CS018205000001,P071401020,2200


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
> 
> `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [13]:
@time \
@rsubset df_store !(:prefecture_cd == "13" || :floor_area > 900)

  0.000002 seconds


Row,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String,String31,String,String15,String,String,String,Float64,Float64,Float64
1,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.592,35.5619,831.0
2,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.632,35.5466,890.0
3,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.022,35.6612,808.0


In [14]:
@time \
@rsubset df_store :prefecture_cd != "13" && :floor_area <= 900

  0.000001 seconds


Row,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String,String31,String,String15,String,String,String,Float64,Float64,Float64
1,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.592,35.5619,831.0
2,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.632,35.5466,890.0
3,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.022,35.6612,808.0


---
> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [15]:
@time \
@chain df_store begin
    @rsubset startswith(:store_cd, "S14")
    first(10)
end

  0.000001 seconds


Row,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String,String31,String,String15,String,String,String,Float64,Float64,Float64
1,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.633,35.5005,1732.0
2,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.496,35.4592,1495.0
3,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.372,35.5733,1679.0
4,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.499,35.524,1548.0
5,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.496,35.4592,1830.0
6,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.496,35.463,1574.0
7,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.658,35.4216,1341.0
8,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.592,35.5619,831.0
9,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.579,35.2964,1838.0


---
> P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [16]:
@time \
@chain df_customer begin
    @rsubset endswith(:customer_id, "1")
    first(10)
end

  0.000005 seconds


Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String,String31,String,String7,Date,Int64,String,String,String,String,String
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
3,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
4,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
5,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
6,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
7,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
8,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
9,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0


---
> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [17]:
@time \
@rsubset df_store contains(:address, "横浜市")

  0.000001 seconds


Row,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String,String31,String,String15,String,String,String,Float64,Float64,Float64
1,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.633,35.5005,1732.0
2,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.496,35.4592,1495.0
3,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.499,35.524,1548.0
4,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.496,35.4592,1830.0
5,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.496,35.463,1574.0
6,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.658,35.4216,1341.0
7,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.592,35.5619,831.0
8,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.632,35.5466,890.0
9,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.576,35.5491,1657.0


---
> P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [18]:
@time \
@chain df_customer begin
    @rsubset startswith(:status_cd, r"[A-F]")
    first(10)
end

  0.000001 seconds


Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String,String31,String,String7,Date,Int64,String,String,String,String,String
1,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
2,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
3,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
4,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
5,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
6,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
7,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
8,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
9,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E


---
> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [19]:
@time \
@chain df_customer begin
    @rsubset endswith(:status_cd, r"[1-9]")
    first(10)
end

  0.000002 seconds


Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String,String31,String,String7,Date,Int64,String,String,String,String,String
1,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
2,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
3,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
4,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
5,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
6,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
7,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
8,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
9,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1


---
> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [20]:
@time \
@chain df_customer begin
    @rsubset occursin(r"^[A-F].*[1-9]$", :status_cd)
    first(10)
end

  0.000001 seconds


Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String,String31,String,String7,Date,Int64,String,String,String,String,String
1,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
2,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
3,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
4,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
5,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
6,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
7,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
8,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
9,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1


---
> P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [21]:
@time \
@rsubset df_store occursin(r"^[0-9]{3}-[0-9]{3}-[0-9]{4}$", :tel_no)

  0.000001 seconds


Row,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String,String31,String,String15,String,String,String,Float64,Float64,Float64
1,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.6356,1698.0
2,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.48,35.7057,1735.0
3,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.633,35.5005,1732.0
4,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.496,35.4592,1495.0
5,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.372,35.5733,1679.0
6,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.499,35.524,1548.0
7,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.496,35.4592,1830.0
8,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.438,35.5529,1087.0
9,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.496,35.463,1574.0


---
> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [22]:
@time \
@chain df_customer begin
    @orderby :birth_day
    first(10)
end

  0.000001 seconds


Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String,String31,String,String7,Date,Int64,String,String,String,String,String
1,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
2,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
3,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
4,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
5,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
6,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
7,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
8,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
9,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


---
> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [23]:
@time \
@chain df_customer begin
    @orderby ordinalrank(:birth_day, rev=true)
    first(10)
end

  0.000001 seconds


Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String,String31,String,String7,Date,Int64,String,String,String,String,String
1,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
2,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
3,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
4,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
5,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
6,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
7,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
8,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
9,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F


---
> P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [24]:
@time \
@chain df_receipt begin
    @transform :rank = competerank(:amount, rev=true)
    @select :customer_id :amount :rank
    @orderby :rank
    first(10)
end

  0.000001 seconds


Row,customer_id,amount,rank
,String,Int64,Int64
1,CS011415000006,10925,1
2,ZZ000000000000,6800,2
3,CS028605000002,5780,3
4,CS015515000034,5480,4
5,ZZ000000000000,5480,4
6,ZZ000000000000,5480,4
7,ZZ000000000000,5440,7
8,CS021515000089,5440,7
9,CS015515000083,5280,9


---
> P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [25]:
@time \
@chain df_receipt begin
    @transform :rank = ordinalrank(:amount, rev=true)
    @select :customer_id :amount :rank
    @orderby :rank
    first(10)
end

  0.000002 seconds


Row,customer_id,amount,rank
,String,Int64,Int64
1,CS011415000006,10925,1
2,ZZ000000000000,6800,2
3,CS028605000002,5780,3
4,CS015515000034,5480,4
5,ZZ000000000000,5480,5
6,ZZ000000000000,5480,6
7,ZZ000000000000,5440,7
8,CS021515000089,5440,8
9,CS015515000083,5280,9


---
> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [26]:
@time \
nrow(df_receipt)

  0.000001 seconds


104681

---
> P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [27]:
@time \
length(unique(df_receipt.customer_id))

  0.000006 seconds


8307

---
> P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [28]:
@time \
@chain df_receipt begin
    groupby(:store_cd, sort=true)
    @combine begin
        :amount_sum = sum(:amount)
        :quantity_sum = sum(:quantity)
    end
end

  0.000003 seconds


Row,store_cd,amount_sum,quantity_sum
,String,Int64,Int64
1,S12007,638761,2099
2,S12013,787513,2425
3,S12014,725167,2358
4,S12029,794741,2555
5,S12030,684402,2403
6,S13001,811936,2347
7,S13002,727821,2340
8,S13003,764294,2197
9,S13004,779373,2390


---
> P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [29]:
@time \
@chain df_receipt begin
    groupby(:customer_id, sort=true)
    @combine :sales_ymd_latest = maximum(:sales_ymd)
    first(10)
end

  0.000001 seconds


Row,customer_id,sales_ymd_latest
,String,Int64
1,CS001113000004,20190308
2,CS001114000005,20190731
3,CS001115000010,20190405
4,CS001205000004,20190625
5,CS001205000006,20190224
6,CS001211000025,20190322
7,CS001212000027,20170127
8,CS001212000031,20180906
9,CS001212000046,20170811


---
> P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [30]:
@time \
@chain df_receipt begin
    groupby(:customer_id, sort=true)
    @combine :sales_ymd_first = minimum(:sales_ymd)
    first(10)
end

  0.000006 seconds


Row,customer_id,sales_ymd_first
,String,Int64
1,CS001113000004,20190308
2,CS001114000005,20180503
3,CS001115000010,20171228
4,CS001205000004,20170914
5,CS001205000006,20180207
6,CS001211000025,20190322
7,CS001212000027,20170127
8,CS001212000031,20180906
9,CS001212000046,20170811


---
> P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [31]:
@time \
@chain df_receipt begin
    groupby(:customer_id, sort=true)
    @combine begin
        :sales_ymd_latest = maximum(:sales_ymd)
        :sales_ymd_first = minimum(:sales_ymd)
    end
    @rsubset :sales_ymd_latest != :sales_ymd_first
    first(10)
end

  0.000002 seconds


Row,customer_id,sales_ymd_latest,sales_ymd_first
,String,Int64,Int64
1,CS001114000005,20190731,20180503
2,CS001115000010,20190405,20171228
3,CS001205000004,20190625,20170914
4,CS001205000006,20190224,20180207
5,CS001214000009,20190902,20170306
6,CS001214000017,20191006,20180828
7,CS001214000048,20190929,20171109
8,CS001214000052,20190617,20180208
9,CS001215000005,20181021,20170206


---
> P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [32]:
@time \
@chain df_receipt begin
    groupby(:store_cd)
    @combine :amount_mean = mean(:amount)
    @orderby -:amount_mean
    first(5)
end

  0.000001 seconds


Row,store_cd,amount_mean
,String,Float64
1,S13052,402.867
2,S13015,351.112
3,S13003,350.916
4,S14010,348.791
5,S13001,348.47


---
> P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [33]:
@time \
@chain df_receipt begin
    groupby(:store_cd)
    @combine :amount_median = median(:amount)
    @orderby -:amount_median
    first(5)
end

  0.000001 seconds


Row,store_cd,amount_median
,String,Float64
1,S13052,190.0
2,S14010,188.0
3,S14050,185.0
4,S13003,180.0
5,S14040,180.0


---
> P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [34]:
@time \
@chain df_receipt begin
    groupby(:store_cd, sort=true)
    @combine :product_cd_mode = mode(:product_cd)
    first(10)
end

  0.000005 seconds


Row,store_cd,product_cd_mode
,String,String
1,S12007,P060303001
2,S12013,P060303001
3,S12014,P060303001
4,S12029,P060303001
5,S12030,P060303001
6,S13001,P060303001
7,S13002,P060303001
8,S13003,P071401001
9,S13004,P060303001


---
> P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [35]:
@time \
@chain df_receipt begin
    groupby(:store_cd)
    @combine :amount_var = var(:amount, corrected=false)
    @orderby -:amount_var
    first(5)
end

  0.000001 seconds


Row,store_cd,amount_var
,String,Float64
1,S13052,4.40089e5
2,S14011,3.06315e5
3,S14034,2.9692e5
4,S13001,295432.0
5,S13015,2.95294e5


---
> P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

In [36]:
@time \
@chain df_receipt begin
    groupby(:store_cd)
    @combine :amount_sd = std(:amount, corrected=false)
    @orderby -:amount_sd
    first(5)
end

  0.000005 seconds


Row,store_cd,amount_sd
,String,Float64
1,S13052,663.392
2,S14011,553.457
3,S14034,544.904
4,S13001,543.537
5,S13015,543.41


---
> P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [37]:
@time \
percentile(df_receipt.amount, [25, 50, 75, 100])

  0.000001 seconds


4-element Vector{Float64}:
   102.0
   170.0
   288.0
 10925.0

---
> P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [38]:
@time \
@chain df_receipt begin
    groupby(:store_cd, sort=true)
    @combine :amount_mean = mean(:amount)
    @rsubset :amount_mean >= 330
end

  0.000001 seconds


Row,store_cd,amount_mean
,String,Float64
1,S12013,330.194
2,S13001,348.47
3,S13003,350.916
4,S13004,330.944
5,S13015,351.112
6,S13019,330.209
7,S13020,337.88
8,S13052,402.867
9,S14010,348.791


---
> P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [39]:
@time \
@chain df_receipt begin
    @rsubset !startswith(:customer_id, "Z")
    groupby(:customer_id, sort=true)
    @combine :amount_sum = sum(:amount)
    @combine :amount_sum_mean = mean(:amount_sum)
end

  0.000001 seconds


Row,amount_sum_mean
,Float64
1,2547.74


---
> P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [40]:
@time \
@chain df_receipt begin
    @rsubset !startswith(:customer_id, "Z")
    groupby(:customer_id, sort=true)
    @combine :amount_sum = sum(:amount)
    @transform :amount_sum_mean = mean(:amount_sum)
    @rsubset :amount_sum >= :amount_sum_mean
    first(10)
end

  0.000001 seconds


Row,customer_id,amount_sum,amount_sum_mean
,String,Int64,Float64
1,CS001115000010,3044,2547.74
2,CS001205000006,3337,2547.74
3,CS001214000009,4685,2547.74
4,CS001214000017,4132,2547.74
5,CS001214000052,5639,2547.74
6,CS001215000040,3496,2547.74
7,CS001304000006,3726,2547.74
8,CS001305000005,3485,2547.74
9,CS001305000011,4370,2547.74


---
> P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [41]:
@time \
@chain df_receipt begin
    innerjoin(df_store[!, [:store_cd, :store_name]], on=:store_cd)
    first(10)
end

  0.000002 seconds


Row,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
,Int64,Int64,String,Int64,Int64,String,String,Int64,Int64,String31
1,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
2,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81,成城店
3,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170,二ツ橋店
4,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25,新山下店
5,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90,大和店
6,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138,狛江店
7,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30,三田店
8,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128,長津田店
9,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770,十条仲原店


---
> P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [42]:
@time \
@chain df_product begin
    innerjoin(
        df_category[!, [:category_small_cd, :category_small_name]],
        on=:category_small_cd
    )
    first(10)
end

  0.000001 seconds


Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
,String,String,String,String,Int64?,Int64?,String
1,P040101001,04,0401,040101,198,149,弁当類
2,P040101002,04,0401,040101,218,164,弁当類
3,P040101003,04,0401,040101,230,173,弁当類
4,P040101004,04,0401,040101,248,186,弁当類
5,P040101005,04,0401,040101,268,201,弁当類
6,P040101006,04,0401,040101,298,224,弁当類
7,P040101007,04,0401,040101,338,254,弁当類
8,P040101008,04,0401,040101,420,315,弁当類
9,P040101009,04,0401,040101,498,374,弁当類


---
> P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [43]:
@time \
@chain df_customer begin
    @rsubset :gender_cd == "1" && !startswith(:customer_id, "Z")
    leftjoin(df_receipt[!, [:customer_id, :amount]], on=:customer_id)
    groupby(:customer_id, sort=true)
    @combine :amount = coalesce.(sum(:amount), 0)
    first(10)
end

  0.000001 seconds


Row,customer_id,amount
,String,Int64
1,CS001112000009,0
2,CS001112000019,0
3,CS001112000021,0
4,CS001112000023,0
5,CS001112000024,0
6,CS001112000029,0
7,CS001112000030,0
8,CS001113000004,1298
9,CS001113000010,0


---
> P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [44]:
function p039()
    gdf = @chain df_receipt begin
        @rsubset !startswith(:customer_id, "Z")
        groupby(:customer_id)
    end

    df1 = @chain gdf begin
        @combine :sales_ymd_count = length(unique(:sales_ymd))
        @orderby -:sales_ymd_count
        first(20)
    end

    df2 = @chain gdf begin
        @combine :amount_sum = sum(:amount)
        @orderby -:amount_sum
        first(20)
    end

    @chain df1 begin
        outerjoin(df2, on=:customer_id)
        @orderby(-:sales_ymd_count)
    end
end

@time p039()

  0.765621 seconds (980.27 k allocations: 62.566 MiB, 97.29% compilation time)


Row,customer_id,sales_ymd_count,amount_sum
,String,Int64?,Int64?
1,CS040214000008,23,missing
2,CS015415000185,22,20153
3,CS010214000010,22,18585
4,CS028415000007,21,19127
5,CS010214000002,21,missing
6,CS017415000097,20,23086
7,CS016415000141,20,18372
8,CS031414000051,19,19202
9,CS039414000052,19,missing


---
> P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [45]:
@time \
crossjoin(df_store, df_product) |> nrow

  0.000001 seconds


531590

---
> P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [46]:
@time \
@chain df_receipt begin
    groupby(:sales_ymd, sort=true)
    @combine :amount_sum = sum(:amount)
    @transform @astable begin
        :sales_ymd_lag = lag(:sales_ymd)
        :amount_sum_lag = lag(:amount_sum)
        :amount_diff = :amount_sum - :amount_sum_lag
    end
    first(10)
end

  0.000001 seconds


Row,sales_ymd,amount_sum,sales_ymd_lag,amount_sum_lag,amount_diff
,Int64,Int64,Int64?,Int64?,Int64?
1,20170101,33723,missing,missing,missing
2,20170102,24165,20170101,33723,-9558
3,20170103,27503,20170102,24165,3338
4,20170104,36165,20170103,27503,8662
5,20170105,37830,20170104,36165,1665
6,20170106,32387,20170105,37830,-5443
7,20170107,23415,20170106,32387,-8972
8,20170108,24737,20170107,23415,1322
9,20170109,26718,20170108,24737,1981


---
> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [47]:
@time \
@chain df_receipt begin
    groupby(:sales_ymd, sort=true)
    @combine :amount_sum = sum(:amount)
    @transform begin
        :sales_ymd_lag1 = lag(:sales_ymd, 1)
        :amount_sum_lag1 = lag(:amount_sum, 1)
        :sales_ymd_lag2 = lag(:sales_ymd, 2)
        :amount_sum_lag2 = lag(:amount_sum, 2)
        :sales_ymd_lag3 = lag(:sales_ymd, 3)
        :amount_sum_lag3 = lag(:amount_sum, 3)
    end
    first(10)
end

  0.000001 seconds


Row,sales_ymd,amount_sum,sales_ymd_lag1,amount_sum_lag1,sales_ymd_lag2,amount_sum_lag2,sales_ymd_lag3,amount_sum_lag3
,Int64,Int64,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?
1,20170101,33723,missing,missing,missing,missing,missing,missing
2,20170102,24165,20170101,33723,missing,missing,missing,missing
3,20170103,27503,20170102,24165,20170101,33723,missing,missing
4,20170104,36165,20170103,27503,20170102,24165,20170101,33723
5,20170105,37830,20170104,36165,20170103,27503,20170102,24165
6,20170106,32387,20170105,37830,20170104,36165,20170103,27503
7,20170107,23415,20170106,32387,20170105,37830,20170104,36165
8,20170108,24737,20170107,23415,20170106,32387,20170105,37830
9,20170109,26718,20170108,24737,20170107,23415,20170106,32387


---
> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [48]:
@time \
@chain df_receipt begin
    innerjoin(df_customer[!, [:customer_id, :gender_cd, :age]], on=:customer_id)
    @rtransform begin
        :gender = :gender_cd == "0" ? "Male" :
                  :gender_cd == "1" ? "Female" :
                  :gender_cd == "9" ? "Unknown" : error("Invalid gender code: $(:gender_cd)")
        :age_cat = floor(Int, :age / 10) * 10
    end
    groupby([:age_cat, :gender_cd, :gender], sort=true)
    @combine :amount_sum = sum(:amount)
    unstack(:age_cat, :gender, :amount_sum)
end

  0.000001 seconds


Row,age_cat,Male,Female,Unknown
,Int64,Int64?,Int64?,Int64?
1,10,1591,149836,4317
2,20,72940,1363724,44328
3,30,177322,693047,50441
4,40,19355,9320791,483512
5,50,54320,6685192,342923
6,60,272469,987741,71418
7,70,13435,29764,2427
8,80,46360,262923,5111
9,90,missing,6260,missing


In [49]:
function format_gender(code)
    code == "0" && return "male"
    code == "1" && return "female"
    code == "9" && return "unknown"
    error("Invalid gender code: $(code)")
end

function p043()
    @chain df_receipt begin
        innerjoin(df_customer[!, [:customer_id, :gender_cd, :age]], on=:customer_id)
        @rtransform begin
            :gender = format_gender(:gender_cd)
            :age_cat = floor(Int, :age / 10) * 10
        end
        groupby([:age_cat, :gender_cd, :gender], sort=true)
        @combine :amount_sum = sum(:amount)
        unstack(:age_cat, :gender, :amount_sum)
    end
end

@time p043()

  0.133255 seconds (132.55 k allocations: 23.592 MiB, 84.60% compilation time)


Row,age_cat,male,female,unknown
,Int64,Int64?,Int64?,Int64?
1,10,1591,149836,4317
2,20,72940,1363724,44328
3,30,177322,693047,50441
4,40,19355,9320791,483512
5,50,54320,6685192,342923
6,60,272469,987741,71418
7,70,13435,29764,2427
8,80,46360,262923,5111
9,90,missing,6260,missing


---
> P-044： 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [50]:
@time \
@chain p043() begin
    stack([:male, :female, :unknown], variable_name=:gender, value_name=:amount)
    dropmissing(:amount)
    @rtransform :gender_cd = :gender == "male"    ? "00" :
                             :gender == "female"  ? "01" :
                             :gender == "unknown" ? "09" : error("Invalid gender: $(:gender)")
    @select :age_cat :gender_cd :amount
    @orderby :age_cat :gender_cd
end

  0.000006 seconds


Row,age_cat,gender_cd,amount
,Int64,String,Int64
1,10,00,1591
2,10,01,149836
3,10,09,4317
4,20,00,72940
5,20,01,1363724
6,20,09,44328
7,30,00,177322
8,30,01,693047
9,30,09,50441


---
> P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [51]:
@time \
@chain df_customer begin
    @select :customer_id :birth_day
    @rtransform :birth_day = Dates.format(:birth_day, "yyyymmdd")
    first(10)
end

  0.000001 seconds


Row,customer_id,birth_day
,String,String
1,CS021313000114,19810429
2,CS037613000071,19520401
3,CS031415000172,19761004
4,CS028811000001,19330327
5,CS001215000145,19950329
6,CS020401000016,19740915
7,CS015414000103,19770809
8,CS029403000008,19730817
9,CS015804000004,19310502


---
> P-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [52]:
@time \
@chain df_customer begin
    @select :customer_id :application_date
    @rtransform :application_date = Date(:application_date, "yyyymmdd")
    first(10)
end

  0.000005 seconds


Row,customer_id,application_date
,String,Date
1,CS021313000114,2015-09-05
2,CS037613000071,2015-04-14
3,CS031415000172,2015-05-29
4,CS028811000001,2016-01-15
5,CS001215000145,2017-06-05
6,CS020401000016,2015-02-25
7,CS015414000103,2015-07-22
8,CS029403000008,2015-05-15
9,CS015804000004,2015-06-07


---
> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [53]:
@time \
@chain df_receipt begin
    @select  :receipt_no :receipt_sub_no :sales_ymd
    @rtransform :sales_ymd = Date(string(:sales_ymd), "yyyymmdd")
    first(10)
end

  0.000002 seconds


Row,receipt_no,receipt_sub_no,sales_ymd
,Int64,Int64,Date
1,112,1,2018-11-03
2,1132,2,2018-11-18
3,1102,1,2017-07-12
4,1132,1,2019-02-05
5,1102,2,2018-08-21
6,1112,1,2019-06-05
7,1102,2,2018-12-05
8,1102,1,2019-09-22
9,1112,2,2017-05-04


---
> P-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [54]:
unix2date(x) = Date(unix2datetime(x))

@time \
@chain df_receipt begin
    @select :receipt_no :receipt_sub_no :sales_epoch
    @rtransform :sales_ymd = unix2date(:sales_epoch)
    @select $(Not(:sales_epoch))
    first(10)
end

  0.000001 seconds


Row,receipt_no,receipt_sub_no,sales_ymd
,Int64,Int64,Date
1,112,1,2018-11-03
2,1132,2,2018-11-18
3,1102,1,2017-07-12
4,1132,1,2019-02-05
5,1102,2,2018-08-21
6,1112,1,2019-06-05
7,1102,2,2018-12-05
8,1102,1,2019-09-22
9,1112,2,2017-05-04


---
> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [55]:
@time \
@chain df_receipt begin
    @select :receipt_no :receipt_sub_no :sales_epoch
    @rtransform :sales_y = year(unix2date(:sales_epoch))
    @select $(Not(:sales_epoch))
    first(10)
end

  0.000001 seconds


Row,receipt_no,receipt_sub_no,sales_y
,Int64,Int64,Int64
1,112,1,2018
2,1132,2,2018
3,1102,1,2017
4,1132,1,2019
5,1102,2,2018
6,1112,1,2019
7,1102,2,2018
8,1102,1,2019
9,1112,2,2017


---
> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [56]:
@time \
@chain df_receipt begin
    @select :receipt_no :receipt_sub_no :sales_epoch
    @rtransform :sales_m = Dates.format(unix2date(:sales_epoch), "mm")
    @select $(Not(:sales_epoch))
    first(10)
end

  0.000002 seconds


Row,receipt_no,receipt_sub_no,sales_m
,Int64,Int64,String
1,112,1,11
2,1132,2,11
3,1102,1,07
4,1132,1,02
5,1102,2,08
6,1112,1,06
7,1102,2,12
8,1102,1,09
9,1112,2,05


---
> P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [57]:
@time \
@chain df_receipt begin
    @select :receipt_no :receipt_sub_no :sales_epoch
    @rtransform :sales_d = Dates.format(unix2date(:sales_epoch), "dd")
    @select $(Not(:sales_epoch))
    first(10)
end

  0.000002 seconds


Row,receipt_no,receipt_sub_no,sales_d
,Int64,Int64,String
1,112,1,03
2,1132,2,18
3,1102,1,12
4,1132,1,05
5,1102,2,21
6,1112,1,05
7,1102,2,05
8,1102,1,22
9,1112,2,04


---
> P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [58]:
@time \
@chain df_receipt begin
    @rsubset !startswith(:customer_id, "Z")
    groupby(:customer_id, sort=true)
    @combine :amount_sum = sum(:amount)
    @rtransform :amount_cat = ifelse(:amount_sum <= 2000, 0, 1)
    first(10)
end

  0.000002 seconds


Row,customer_id,amount_sum,amount_cat
,String,Int64,Int64
1,CS001113000004,1298,0
2,CS001114000005,626,0
3,CS001115000010,3044,1
4,CS001205000004,1988,0
5,CS001205000006,3337,1
6,CS001211000025,456,0
7,CS001212000027,448,0
8,CS001212000031,296,0
9,CS001212000046,228,0


---
> P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [59]:
@time \
@chain df_customer begin
    @select :customer_id :postal_cd
    @rtransform :is_tokyo = ifelse(100 <= parse(Int, :postal_cd[1:3]) <= 209, 1, 0)
    innerjoin(unique(df_receipt[!, [:customer_id]]), on=:customer_id)
    groupby(:is_tokyo)
    @combine :n_customer = length(:customer_id)
end

  0.000001 seconds


Row,is_tokyo,n_customer
,Int64,Int64
1,0,3906
2,1,4400


---
> P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [60]:
@time \
@chain df_customer begin
    @select :customer_id :address
    @rtransform :address_cd = occursin("埼玉県", :address) ? 11 :
                              occursin("千葉県", :address) ? 12 :
                              occursin("東京都", :address) ? 13 :
                              occursin("神奈川県", :address) ? 14 :
                              error("Invarid address: $(:address)")
    first(10)
end

  0.000005 seconds


Row,customer_id,address,address_cd
,String,String,Int64
1,CS021313000114,神奈川県伊勢原市粟窪**********,14
2,CS037613000071,東京都江東区南砂**********,13
3,CS031415000172,東京都渋谷区代々木**********,13
4,CS028811000001,神奈川県横浜市泉区和泉町**********,14
5,CS001215000145,東京都大田区仲六郷**********,13
6,CS020401000016,東京都板橋区若木**********,13
7,CS015414000103,東京都江東区北砂**********,13
8,CS029403000008,千葉県浦安市海楽**********,12
9,CS015804000004,東京都江東区北砂**********,13


---
> P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [61]:
@time \
@chain df_receipt begin
    groupby(:customer_id, sort=true)
    @combine :amount_sum = sum(:amount)
    @transform begin
        :q1 = quantile(:amount_sum, 0.25)
        :q2 = quantile(:amount_sum, 0.5)
        :q3 = quantile(:amount_sum, 0.75)
    end
    @rtransform :amount_cat = :amount_sum < :q1 ? 1 :
                              :amount_sum < :q2 ? 2 :
                              :amount_sum < :q3 ? 3 : 4
    @orderby :customer_id
    first(10)
end

  0.000001 seconds


Row,customer_id,amount_sum,q1,q2,q3,amount_cat
,String,Int64,Float64,Float64,Float64,Int64
1,CS001113000004,1298,548.5,1478.0,3651.0,2
2,CS001114000005,626,548.5,1478.0,3651.0,2
3,CS001115000010,3044,548.5,1478.0,3651.0,3
4,CS001205000004,1988,548.5,1478.0,3651.0,3
5,CS001205000006,3337,548.5,1478.0,3651.0,3
6,CS001211000025,456,548.5,1478.0,3651.0,1
7,CS001212000027,448,548.5,1478.0,3651.0,1
8,CS001212000031,296,548.5,1478.0,3651.0,1
9,CS001212000046,228,548.5,1478.0,3651.0,1


---
> P-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [62]:
@time \
 @chain df_customer begin
    @rtransform :age_cat = ifelse(:age >= 60, 60, floor(Int, :age / 10) * 10)
    @select :customer_id :birth_day :age_cat
    first(10)
end

  0.000001 seconds


Row,customer_id,birth_day,age_cat
,String,Date,Int64
1,CS021313000114,1981-04-29,30
2,CS037613000071,1952-04-01,60
3,CS031415000172,1976-10-04,40
4,CS028811000001,1933-03-27,60
5,CS001215000145,1995-03-29,20
6,CS020401000016,1974-09-15,40
7,CS015414000103,1977-08-09,40
8,CS029403000008,1973-08-17,40
9,CS015804000004,1931-05-02,60


---
> P-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [63]:
function format_gender_agecat(gender, agecat)
    g = format_gender(gender) |> titlecase
    a = ifelse(agecat == 60, ">=", "") * string(agecat)
    "$(g): $(a)s"
end

@time \
 @chain df_customer begin
    @rtransform @astable begin
        :age_cat = ifelse(:age >= 60, 60, floor(Int, :age / 10) * 10)
        :gender_age_cat = format_gender_agecat(:gender_cd, :age_cat)
    end
    @select :customer_id :birth_day :age_cat :gender_cd :gender_age_cat
    first(10)
end

  0.000001 seconds


Row,customer_id,birth_day,age_cat,gender_cd,gender_age_cat
,String,Date,Int64,String,String
1,CS021313000114,1981-04-29,30,1,Female: 30s
2,CS037613000071,1952-04-01,60,9,Unknown: >=60s
3,CS031415000172,1976-10-04,40,1,Female: 40s
4,CS028811000001,1933-03-27,60,1,Female: >=60s
5,CS001215000145,1995-03-29,20,1,Female: 20s
6,CS020401000016,1974-09-15,40,0,Male: 40s
7,CS015414000103,1977-08-09,40,1,Female: 40s
8,CS029403000008,1973-08-17,40,0,Male: 40s
9,CS015804000004,1931-05-02,60,0,Male: >=60s


---
> P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [64]:
# !!!
@time \
@chain df_customer begin
    @select :customer_id :gender_cd
    @rtransform begin
        :gender_cd_0 = ifelse(:gender_cd == "0", 1, 0)
        :gender_cd_1 = ifelse(:gender_cd == "1", 1, 0)
        :gender_cd_9 = ifelse(:gender_cd == "9", 1, 0)
    end
    first(10)
end

  0.000001 seconds


Row,customer_id,gender_cd,gender_cd_0,gender_cd_1,gender_cd_9
,String,String,Int64,Int64,Int64
1,CS021313000114,1,0,1,0
2,CS037613000071,9,0,0,1
3,CS031415000172,1,0,1,0
4,CS028811000001,1,0,1,0
5,CS001215000145,1,0,1,0
6,CS020401000016,0,1,0,0
7,CS015414000103,1,0,1,0
8,CS029403000008,0,1,0,0
9,CS015804000004,0,1,0,0


---
> P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [65]:
@time \
@chain df_receipt begin
    @rsubset !startswith(:customer_id, "Z")
    groupby(:customer_id, sort=true)
    @combine :amount_sum = sum(:amount)
    @transform :amount_sum_std = zscore(:amount_sum, mean(:amount_sum), std(:amount_sum))
    first(10)
end

  0.000005 seconds


Row,customer_id,amount_sum,amount_sum_std
,String,Int64,Float64
1,CS001113000004,1298,-0.45935
2,CS001114000005,626,-0.706348
3,CS001115000010,3044,0.182403
4,CS001205000004,1988,-0.205737
5,CS001205000006,3337,0.290096
6,CS001211000025,456,-0.768832
7,CS001212000027,448,-0.771773
8,CS001212000031,296,-0.827641
9,CS001212000046,228,-0.852635


---
> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [66]:
function min_max_normalize(x)
    min_x = minimum(x)
    max_x = maximum(x)
    @. (x - min_x) / (max_x - min_x)
end

@time \
@chain df_receipt begin
    @rsubset !startswith(:customer_id, "Z")
    groupby(:customer_id, sort=true)
    @combine :amount_sum = sum(:amount)
    @transform :amount_sum_converted = min_max_normalize(:amount_sum)
    @select :customer_id :amount_sum :amount_sum_converted
    first(10)
end

  0.000006 seconds


Row,customer_id,amount_sum,amount_sum_converted
,String,Int64,Float64
1,CS001113000004,1298,0.0533542
2,CS001114000005,626,0.0241571
3,CS001115000010,3044,0.129214
4,CS001205000004,1988,0.0833333
5,CS001205000006,3337,0.141945
6,CS001211000025,456,0.0167709
7,CS001212000027,448,0.0164234
8,CS001212000031,296,0.00981926
9,CS001212000046,228,0.00686479


---
> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [67]:
@time \
@chain df_receipt begin
    @rsubset !startswith(:customer_id, "Z")
    groupby(:customer_id, sort=true)
    @combine :amount_sum_log = log10(sum(:amount))
    first(10)
end

  0.000001 seconds


Row,customer_id,amount_sum_log
,String,Float64
1,CS001113000004,3.11327
2,CS001114000005,2.79657
3,CS001115000010,3.48344
4,CS001205000004,3.29842
5,CS001205000006,3.52336
6,CS001211000025,2.65896
7,CS001212000027,2.65128
8,CS001212000031,2.47129
9,CS001212000046,2.35793


---
> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [68]:
@time \
@chain df_receipt begin
    @rsubset !startswith(:customer_id, "Z")
    groupby(:customer_id, sort=true)
    @combine :amount_sum = sum(:amount)
    @rtransform :amount_sum_log = log(:amount_sum)
    first(10)
end

  0.000001 seconds


Row,customer_id,amount_sum,amount_sum_log
,String,Int64,Float64
1,CS001113000004,1298,7.16858
2,CS001114000005,626,6.43935
3,CS001115000010,3044,8.02093
4,CS001205000004,1988,7.59488
5,CS001205000006,3337,8.11283
6,CS001211000025,456,6.12249
7,CS001212000027,448,6.10479
8,CS001212000031,296,5.69036
9,CS001212000046,228,5.42935


---
> P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [69]:
@time \
@chain df_product begin
    @rtransform :profit = :unit_price - :unit_cost
    first(10)
end

  0.000005 seconds


Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,profit
,String,String,String,String,Int64?,Int64?,Int64?
1,P040101001,04,0401,040101,198,149,49
2,P040101002,04,0401,040101,218,164,54
3,P040101003,04,0401,040101,230,173,57
4,P040101004,04,0401,040101,248,186,62
5,P040101005,04,0401,040101,268,201,67
6,P040101006,04,0401,040101,298,224,74
7,P040101007,04,0401,040101,338,254,84
8,P040101008,04,0401,040101,420,315,105
9,P040101009,04,0401,040101,498,374,124


---
> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [70]:
@time \
@chain df_product begin
    @select :product_cd :unit_price :unit_cost
    @rtransform :profit_ratio = (:unit_price - :unit_cost) / :unit_price
    @combine :profit_ratio_mean = mean(skipmissing(:profit_ratio))
end

  0.000005 seconds


Row,profit_ratio_mean
,Float64
1,0.249114


---
> P-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

$$
r = (p-c)/p \\
rp = p-c \\
p-rp=c \\
p=c/(1-r)
$$

In [71]:
@time \
@chain df_product begin
    @select :product_cd :unit_price :unit_cost
    @rtransform @astable begin
        :unit_price_new = floor(:unit_cost / (1 - 0.3))
        :profit_ratio_new = (:unit_price_new - :unit_cost) / :unit_price_new
    end
    first(10)
end

  0.000002 seconds


Row,product_cd,unit_price,unit_cost,unit_price_new,profit_ratio_new
,String,Int64?,Int64?,Float64?,Float64?
1,P040101001,198,149,212.0,0.29717
2,P040101002,218,164,234.0,0.299145
3,P040101003,230,173,247.0,0.299595
4,P040101004,248,186,265.0,0.298113
5,P040101005,268,201,287.0,0.299652
6,P040101006,298,224,320.0,0.3
7,P040101007,338,254,362.0,0.298343
8,P040101008,420,315,450.0,0.3
9,P040101009,498,374,534.0,0.299625


---
> P-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [72]:
@time \
@chain df_product begin
    @select :product_cd :unit_price :unit_cost
    @rtransform @astable begin
        :unit_price_new = round(:unit_cost / (1 - 0.3))
        :profit_ratio_new = (:unit_price_new - :unit_cost) / :unit_price_new
    end
    first(10)
end

  0.000001 seconds


Row,product_cd,unit_price,unit_cost,unit_price_new,profit_ratio_new
,String,Int64?,Int64?,Float64?,Float64?
1,P040101001,198,149,213.0,0.300469
2,P040101002,218,164,234.0,0.299145
3,P040101003,230,173,247.0,0.299595
4,P040101004,248,186,266.0,0.300752
5,P040101005,268,201,287.0,0.299652
6,P040101006,298,224,320.0,0.3
7,P040101007,338,254,363.0,0.300275
8,P040101008,420,315,450.0,0.3
9,P040101009,498,374,534.0,0.299625


---
> P-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [73]:
@time \
@chain df_product begin
    @select :product_cd :unit_price :unit_cost
    @rtransform @astable begin
        :unit_price_new = ceil(:unit_cost / (1 - 0.3))
        :profit_ratio_new = (:unit_price_new - :unit_cost) / :unit_price_new
    end
    first(10)
end

  0.000001 seconds


Row,product_cd,unit_price,unit_cost,unit_price_new,profit_ratio_new
,String,Int64?,Int64?,Float64?,Float64?
1,P040101001,198,149,213.0,0.300469
2,P040101002,218,164,235.0,0.302128
3,P040101003,230,173,248.0,0.302419
4,P040101004,248,186,266.0,0.300752
5,P040101005,268,201,288.0,0.302083
6,P040101006,298,224,320.0,0.3
7,P040101007,338,254,363.0,0.300275
8,P040101008,420,315,451.0,0.301552
9,P040101009,498,374,535.0,0.300935


---
> P-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [74]:
@time \
@chain df_product begin
    @select :product_cd :unit_price :unit_cost
    @rtransform :unit_price_including_tax = floor(Int, coalesce.(:unit_price, 0) * 1.1)
    first(10)
end

  0.000007 seconds


Row,product_cd,unit_price,unit_cost,unit_price_including_tax
,String,Int64?,Int64?,Int64
1,P040101001,198,149,217
2,P040101002,218,164,239
3,P040101003,230,173,253
4,P040101004,248,186,272
5,P040101005,268,201,294
6,P040101006,298,224,327
7,P040101007,338,254,371
8,P040101008,420,315,462
9,P040101009,498,374,547


---
> P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [75]:
function p069()
    df_receipt_product = innerjoin(
        df_receipt[!, [:customer_id, :amount, :product_cd]],
        df_product[!, [:product_cd, :category_major_cd]],
        on=:product_cd
    )

    customers_07 = @chain df_receipt_product begin
        @rsubset :category_major_cd == "07"
        groupby(:customer_id)
        @combine :amount_sum_07 = sum(:amount)
    end

    @chain df_receipt_product begin
        rightjoin(customers_07, on=:customer_id)
        groupby([:customer_id, :amount_sum_07], sort=true)
        @combine :amount_sum = sum(:amount)
        @rtransform :prop = :amount_sum_07 / :amount_sum
        first(10)
    end
end

@time p069()

  2.135036 seconds (2.27 M allocations: 141.462 MiB, 2.24% gc time, 95.03% compilation time: 9% of which was recompilation)


Row,customer_id,amount_sum_07,amount_sum,prop
,String,Int64,Int64,Float64
1,CS001113000004,1298,1298,1.0
2,CS001114000005,486,626,0.776358
3,CS001115000010,2694,3044,0.88502
4,CS001205000004,346,1988,0.174044
5,CS001205000006,2004,3337,0.600539
6,CS001212000027,200,448,0.446429
7,CS001212000031,296,296,1.0
8,CS001212000046,108,228,0.473684
9,CS001212000070,308,456,0.675439


---
> P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [76]:
@time \
@chain df_receipt begin
    @select :customer_id :sales_ymd
    @distinct :customer_id :sales_ymd
    innerjoin(df_customer[!, [:customer_id, :application_date]], on=:customer_id)
    @rtransform @astable begin
        :sales_ymd = Date(string(:sales_ymd), "yyyymmdd")
        :application_date = Date(:application_date, "yyyymmdd")
        :elapsed_days = Dates.value(:sales_ymd - :application_date)
    end
    @orderby(:customer_id, :sales_ymd)
    first(10)
end

  0.000001 seconds


Row,customer_id,sales_ymd,application_date,elapsed_days
,String,Date,Date,Int64
1,CS001113000004,2019-03-08,2015-11-05,1219
2,CS001114000005,2018-05-03,2016-04-12,751
3,CS001114000005,2019-07-31,2016-04-12,1205
4,CS001115000010,2017-12-28,2015-04-17,986
5,CS001115000010,2018-07-01,2015-04-17,1171
6,CS001115000010,2019-04-05,2015-04-17,1449
7,CS001205000004,2017-09-14,2016-06-15,456
8,CS001205000004,2018-08-21,2016-06-15,797
9,CS001205000004,2018-09-04,2016-06-15,811


---
> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [77]:
@time \
@chain df_receipt begin
    @select :customer_id :sales_ymd
    @distinct :customer_id :sales_ymd
    innerjoin(df_customer[!, [:customer_id, :application_date]], on=:customer_id)
    @rtransform @astable begin
        :sales_ymd = Date(string(:sales_ymd), "yyyymmdd")
        :application_date = Date(:application_date, "yyyymmdd")
        :delta = relativedelta(:sales_ymd, :application_date)
        :elapsed_months = :delta.years * 12 + :delta.months
    end
    @select $(Not(:delta))
    @orderby(:customer_id, :sales_ymd)
    first(10)
end

  0.000004 seconds


Row,customer_id,sales_ymd,application_date,elapsed_months
,String,Date,Date,Int64
1,CS001113000004,2019-03-08,2015-11-05,40
2,CS001114000005,2018-05-03,2016-04-12,24
3,CS001114000005,2019-07-31,2016-04-12,39
4,CS001115000010,2017-12-28,2015-04-17,32
5,CS001115000010,2018-07-01,2015-04-17,38
6,CS001115000010,2019-04-05,2015-04-17,47
7,CS001205000004,2017-09-14,2016-06-15,14
8,CS001205000004,2018-08-21,2016-06-15,26
9,CS001205000004,2018-09-04,2016-06-15,26


---
> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [78]:
@time \
@chain df_receipt begin
    @select :customer_id :sales_ymd
    @distinct :customer_id :sales_ymd
    innerjoin(df_customer[!, [:customer_id, :application_date]], on=:customer_id)
    @rtransform @astable begin
        :sales_ymd = Date(string(:sales_ymd), "yyyymmdd")
        :application_date = Date(:application_date, "yyyymmdd")
        :elapsed_years = relativedelta(:sales_ymd, :application_date).years
    end
    @orderby(:customer_id, :sales_ymd)
    first(10)
end

  0.000001 seconds


Row,customer_id,sales_ymd,application_date,elapsed_years
,String,Date,Date,Int64
1,CS001113000004,2019-03-08,2015-11-05,3
2,CS001114000005,2018-05-03,2016-04-12,2
3,CS001114000005,2019-07-31,2016-04-12,3
4,CS001115000010,2017-12-28,2015-04-17,2
5,CS001115000010,2018-07-01,2015-04-17,3
6,CS001115000010,2019-04-05,2015-04-17,3
7,CS001205000004,2017-09-14,2016-06-15,1
8,CS001205000004,2018-08-21,2016-06-15,2
9,CS001205000004,2018-09-04,2016-06-15,2


---
> P-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [79]:
@time \
@chain df_receipt begin
    @select :customer_id :sales_ymd
    @distinct :customer_id :sales_ymd
    innerjoin(df_customer[!, [:customer_id, :application_date]], on=:customer_id)
    @rtransform @astable begin
        :sales_ymd = DateTime(string(:sales_ymd), "yyyymmdd")
        :application_date = DateTime(:application_date, "yyyymmdd")
        :elapsed_epoch = datetime2unix(:sales_ymd) - datetime2unix(:application_date)
    end
    @orderby(:customer_id, :sales_ymd)
    first(10)
end

  0.000001 seconds


Row,customer_id,sales_ymd,application_date,elapsed_epoch
,String,DateTime,DateTime,Float64
1,CS001113000004,2019-03-08T00:00:00,2015-11-05T00:00:00,1.05322e8
2,CS001114000005,2018-05-03T00:00:00,2016-04-12T00:00:00,6.48864e7
3,CS001114000005,2019-07-31T00:00:00,2016-04-12T00:00:00,1.04112e8
4,CS001115000010,2017-12-28T00:00:00,2015-04-17T00:00:00,8.51904e7
5,CS001115000010,2018-07-01T00:00:00,2015-04-17T00:00:00,1.01174e8
6,CS001115000010,2019-04-05T00:00:00,2015-04-17T00:00:00,1.25194e8
7,CS001205000004,2017-09-14T00:00:00,2016-06-15T00:00:00,3.93984e7
8,CS001205000004,2018-08-21T00:00:00,2016-06-15T00:00:00,6.88608e7
9,CS001205000004,2018-09-04T00:00:00,2016-06-15T00:00:00,7.00704e7


---
> P-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [80]:
@time \
@chain df_receipt begin
    @select :customer_id :sales_ymd
    @rtransform @astable begin
        :sales_ymd = Date(string(:sales_ymd), "yyyymmdd")
        :monday = firstdayofweek(:sales_ymd)
        :elapsed_days = Dates.value(:sales_ymd - :monday)
    end
    first(10)
end

  0.000002 seconds


Row,customer_id,sales_ymd,monday,elapsed_days
,String,Date,Date,Int64
1,CS006214000001,2018-11-03,2018-10-29,5
2,CS008415000097,2018-11-18,2018-11-12,6
3,CS028414000014,2017-07-12,2017-07-10,2
4,ZZ000000000000,2019-02-05,2019-02-04,1
5,CS025415000050,2018-08-21,2018-08-20,1
6,CS003515000195,2019-06-05,2019-06-03,2
7,CS024514000042,2018-12-05,2018-12-03,2
8,CS040415000178,2019-09-22,2019-09-16,6
9,ZZ000000000000,2017-05-04,2017-05-01,3


---
> P-075: 顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [81]:
function p075()
    n = nrow(df_customer)
    sample_idx = sample(1:n, floor(Int, 1 * n / 100), replace=false)
    df_customer[sample_idx, :]
end

first(p075(), 10)

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String,String31,String,String7,Date,Int64,String,String,String,String,String
1,CS016613000029,上原 咲,1,女性,1953-07-13,65,184-0004,東京都小金井市本町**********,S13016,20150501,0-00000000-0
2,CS002805000005,石川 明慶,0,男性,1932-09-22,86,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20170701,3-20100829-2
3,CS030715000037,柴田 千夏,1,女性,1939-07-03,79,272-0022,千葉県市川市鬼越**********,S12030,20141130,0-00000000-0
4,CS011415000030,川田 鉄二,9,不明,1971-12-01,47,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150422,9-20101021-C
5,CS034515000020,戸塚 優,1,女性,1960-02-21,59,216-0001,神奈川県川崎市宮前区野川**********,S14034,20150630,E-20100805-E
6,CS039511000003,柴田 涼,1,女性,1959-06-06,59,166-0001,東京都杉並区阿佐谷北**********,S13039,20151028,0-00000000-0
7,CS026715000064,杉原 倫子,1,女性,1945-11-29,73,251-0046,神奈川県藤沢市辻堂西海岸**********,S14026,20150629,0-00000000-0
8,CS002503000146,山口 菊生,9,不明,1963-05-15,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20171007,0-00000000-0
9,CS001511000156,本橋 幸子,1,女性,1962-10-28,56,210-0851,神奈川県川崎市川崎区浜町**********,S13001,20170806,0-00000000-0


---
> P-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [82]:
function p076()
    sampled = partition(df_customer.customer_id, 0.1, stratify=parse.(Int, df_customer.gender_cd))[1]
    @chain df_customer begin
        @rsubset :customer_id in sampled
        groupby(:gender_cd, sort=true)
        @combine :count = $nrow
    end
end

@time p076()

  1.833375 seconds (3.83 M allocations: 207.830 MiB, 84.54% compilation time)


Row,gender_cd,count
,String,Int64
1,0,298
2,1,1792
3,9,107


---
> P-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [83]:
@time \
@chain df_receipt begin
    groupby(:customer_id)
    @combine :amount_sum = sum(:amount)
    @rtransform :log_amount_sum = log(:amount_sum)
    @transform @astable begin
        :mean = mean(:log_amount_sum)
        :sd = std(:log_amount_sum)
    end
    @rsubset abs(:log_amount_sum - :mean) > 3 * :sd
    @select :customer_id :amount_sum :log_amount_sum
end

  0.000005 seconds


Row,customer_id,amount_sum,log_amount_sum
,String,Int64,Float64
1,ZZ000000000000,12395003,16.3328


---
> P-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [84]:
quantile(1:10, [0.25, 0.75])

2-element Vector{Float64}:
 3.25
 7.75

In [85]:
@time \
@chain df_receipt begin
    @rsubset !startswith(:customer_id, "Z")
    groupby(:customer_id, sort=true)
    @combine :sum_amount = sum(:amount)
    @transform @astable begin
        :q1 = quantile(:sum_amount, 0.25)
        :q3 = quantile(:sum_amount, 0.75)
        :iqr = iqr(:sum_amount)
    end
    @rsubset :sum_amount < :q1 - 1.5 * :iqr || :sum_amount > :q3 + 1.5 * :iqr
    first(10)
end

  0.000001 seconds


Row,customer_id,sum_amount,q1,q3,iqr
,String,Int64,Float64,Float64,Float64
1,CS001414000048,8584,548.25,3649.75,3101.5
2,CS001605000009,18925,548.25,3649.75,3101.5
3,CS002415000594,9568,548.25,3649.75,3101.5
4,CS004414000181,9584,548.25,3649.75,3101.5
5,CS005415000137,8734,548.25,3649.75,3101.5
6,CS006414000001,9156,548.25,3649.75,3101.5
7,CS006414000029,9179,548.25,3649.75,3101.5
8,CS006415000105,10042,548.25,3649.75,3101.5
9,CS006415000147,12723,548.25,3649.75,3101.5


---
> P-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [86]:
zip(names(df_product), sum.(eachcol(ismissing.(df_product)))) |> collect

6-element Vector{Tuple{String, Int64}}:
 ("product_cd", 0)
 ("category_major_cd", 0)
 ("category_medium_cd", 0)
 ("category_small_cd", 0)
 ("unit_price", 7)
 ("unit_cost", 7)

---
> P-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [87]:
function p080()
    df = dropmissing(df_product)
    (before_n = nrow(df_product), after_n = nrow(df), df = df)
end

@time p080()

  0.338560 seconds (484.81 k allocations: 23.794 MiB, 98.85% compilation time)


(before_n = 10030,
 after_n = 10023,
 df = 10023×6 DataFrame
   Row │ product_cd  category_major_cd  category_medium_cd  category_small_cd  ⋯
       │ String      String             String              String             ⋯
───────┼────────────────────────────────────────────────────────────────────────
     1 │ P040101001  04                 0401                040101             ⋯
     2 │ P040101002  04                 0401                040101
     3 │ P040101003  04                 0401                040101
     4 │ P040101004  04                 0401                040101
     5 │ P040101005  04                 0401                040101             ⋯
     6 │ P040101006  04                 0401                040101
     7 │ P040101007  04                 0401                040101
     8 │ P040101008  04                 0401                040101
     9 │ P040101009  04                 0401                040101             ⋯
    10 │ P040101010  04                 0401       

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [88]:
@time \
@chain df_product begin
    transform(
        [:unit_price, :unit_cost] .=>
        (x -> coalesce.(x, round(Int, mean(skipmissing(x))))) .=>
        [:unit_price_imputed, :unit_cost_imputed]
    )
    @rsubset ismissing(:unit_price)
end

  0.000001 seconds


Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_price_imputed,unit_cost_imputed
,String,String,String,String,Int64?,Int64?,Int64,Int64
1,P040802007,04,0408,040802,missing,missing,403,302
2,P050103021,05,0501,050103,missing,missing,403,302
3,P050405009,05,0504,050405,missing,missing,403,302
4,P060802026,06,0608,060802,missing,missing,403,302
5,P070202092,07,0702,070202,missing,missing,403,302
6,P080504027,08,0805,080504,missing,missing,403,302
7,P090204185,09,0902,090204,missing,missing,403,302


---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [89]:
@time \
@chain df_product begin
    transform(
        [:unit_price, :unit_cost] .=>
        (x -> coalesce.(x, round(Int, median(skipmissing(x))))) .=>
        [:unit_price_imputed, :unit_cost_imputed]
    )
    @rsubset ismissing(:unit_price)
end

  0.000002 seconds


Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_price_imputed,unit_cost_imputed
,String,String,String,String,Int64?,Int64?,Int64,Int64
1,P040802007,04,0408,040802,missing,missing,252,189
2,P050103021,05,0501,050103,missing,missing,252,189
3,P050405009,05,0504,050405,missing,missing,252,189
4,P060802026,06,0608,060802,missing,missing,252,189
5,P070202092,07,0702,070202,missing,missing,252,189
6,P080504027,08,0805,080504,missing,missing,252,189
7,P090204185,09,0902,090204,missing,missing,252,189


---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [90]:
@time \
@chain df_product begin
    groupby(:category_small_cd)
    combine(
        [:unit_price, :unit_cost] .=>
        (x -> round(Int, median(skipmissing(x)))) .=>
        [:unit_price_median, :unit_cost_median]
    )
    leftjoin(df_product, on=:category_small_cd)
    @rtransform begin
        :unit_price_imputed = coalesce(:unit_price, :unit_price_median)
        :unit_cost_imputed = coalesce(:unit_cost, :unit_cost_median)
    end
    @rsubset ismissing(:unit_price)
end

  0.000008 seconds


Row,category_small_cd,unit_price_median,unit_cost_median,product_cd,category_major_cd,category_medium_cd,unit_price,unit_cost,unit_price_imputed,unit_cost_imputed
,String,Int64,Int64,String?,String?,String?,Int64?,Int64?,Int64,Int64
1,040802,313,235,P040802007,04,0408,missing,missing,313,235
2,050103,132,100,P050103021,05,0501,missing,missing,132,100
3,050405,178,134,P050405009,05,0504,missing,missing,178,134
4,060802,270,200,P060802026,06,0608,missing,missing,270,200
5,070202,238,179,P070202092,07,0702,missing,missing,238,179
6,080504,258,196,P080504027,08,0805,missing,missing,258,196
7,090204,694,521,P090204185,09,0902,missing,missing,694,521


---
> P-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [91]:
function p084()
    amount_all = @chain df_receipt begin
        groupby(:customer_id)
        @combine :amount_all = sum(:amount)
    end

    amount_2019 = @chain df_receipt begin
        @rsubset startswith(string(:sales_ymd), "2019")
        groupby(:customer_id)
        @combine :amount_2019 = sum(:amount)
    end

    @chain df_customer begin
        @select :customer_id
        leftjoin(amount_all, on=:customer_id)
        leftjoin(amount_2019, on=:customer_id)
        @rtransform :amount_prop = coalesce(:amount_2019 / :amount_all, 0)
        @rsubset :amount_prop > 0
        first(10)
    end
end

@time p084()

  0.854193 seconds (1.00 M allocations: 61.250 MiB, 95.49% compilation time)


Row,customer_id,amount_all,amount_2019,amount_prop
,String,Int64?,Int64?,Real
1,CS031415000172,5088,2971,0.583923
2,CS015414000103,3122,874,0.279949
3,CS011215000048,3444,248,0.0720093
4,CS029415000023,5167,3767,0.72905
5,CS035415000029,7504,5823,0.775986
6,CS023513000066,771,208,0.26978
7,CS035513000134,1565,463,0.295847
8,CS001515000263,216,216,1.0
9,CS006415000279,229,229,1.0


---
> P-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

In [92]:
function p085()
    @chain df_geocode begin
        groupby(:postal_cd)
        @combine begin
            :longitude = mean(:longitude)
            :latitude = mean(:latitude)
        end
        rightjoin(df_customer, on=:postal_cd)
    end
end

first(p085(), 10)

Row,postal_cd,longitude,latitude,customer_id,customer_name,gender_cd,gender,birth_day,age,address,application_store_cd,application_date,status_cd
,String,Float64?,Float64?,String,String31,String,String7,Date,Int64,String,String,String,String
1,332-0031,139.727,35.8088,CS020301000012,都築 育二,0,男性,1985-08-20,33,埼玉県川口市青木**********,S13020,20161018,0-00000000-0
2,332-0023,139.711,35.7965,CS051412000011,奥村 愛,1,女性,1969-06-04,49,埼玉県川口市飯塚**********,S13051,20180702,0-00000000-0
3,332-0023,139.711,35.7965,CS051412000012,おかやま 未華子,1,女性,1977-03-13,42,埼玉県川口市飯塚**********,S13051,20180511,0-00000000-0
4,332-0023,139.711,35.7965,CS051212000001,美木 瞬,1,女性,1991-10-19,27,埼玉県川口市飯塚**********,S13051,20180509,2-20101018-4
5,332-0015,139.716,35.8023,CS020112000003,荒川 まなみ,1,女性,2004-05-13,14,埼玉県川口市川口**********,S13020,20151116,0-00000000-0
6,332-0015,139.716,35.8023,CS020212000016,藤沢 恵梨香,1,女性,1991-08-08,27,埼玉県川口市川口**********,S13020,20150122,4-20100207-3
7,332-0015,139.716,35.8023,CS051512000001,田原 夏希,1,女性,1959-11-21,59,埼玉県川口市川口**********,S13051,20190325,0-00000000-0
8,332-0015,139.716,35.8023,CS020212000004,前田 美佐,1,女性,1989-12-11,29,埼玉県川口市川口**********,S13020,20150814,C-20090906-9
9,332-0015,139.716,35.8023,CS051502000001,若山 哲平,0,男性,1961-10-07,57,埼玉県川口市川口**********,S13051,20180209,0-00000000-0


---
> P-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\mbox{緯度（ラジアン）}：\phi \\
\mbox{経度（ラジアン）}：\lambda \\
\mbox{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

In [93]:
function distance(latitude, longitude)
    ϕ₁, ϕ₂ = deg2rad.(latitude)
    λ₁, λ₂ = deg2rad.(longitude)
    6371 * acos(sin(ϕ₁) * sin(ϕ₂) + cos(ϕ₁) * cos(ϕ₂) * cos(λ₁ - λ₂))
end

@time \
@chain p085() begin
    innerjoin(df_store, on=:application_store_cd => :store_cd, renamecols="_c" => "_s")
    @rtransform :distance = distance([:latitude_c, :latitude_s], [:longitude_c, :longitude_s])
    @select :customer_id_c :address_c :address_s :distance
    @orderby :customer_id_c
    first(10)
end

  0.000002 seconds


Row,customer_id_c,address_c,address_s,distance
,String,String,String,Float64
1,CS001105000001,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.47979
2,CS001112000009,東京都大田区西馬込**********,東京都大田区仲六郷二丁目,4.02049
3,CS001112000019,東京都大田区昭和島**********,東京都大田区仲六郷二丁目,3.78301
4,CS001112000021,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.47979
5,CS001112000023,東京都大田区昭和島**********,東京都大田区仲六郷二丁目,3.78301
6,CS001112000024,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.47979
7,CS001112000029,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.47979
8,CS001112000030,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.47979
9,CS001113000004,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.47979


---
> P-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [94]:
function p087()
    df_customer_unique = @chain df_customer begin
        leftjoin(df_receipt[!, [:customer_id, :amount]], on=:customer_id)
        groupby(:customer_id)
        @combine :amount_sum = coalesce.(sum(:amount), 0)
        innerjoin(df_customer, on=:customer_id)
        @orderby :customer_name :postal_cd -:amount_sum :customer_id
        groupby([:customer_name, :postal_cd])
        @combine $first 
    end

    count = nrow(df_customer)
    ucount = nrow(df_customer_unique)
    (count = count, unique_count = ucount, diff = count - ucount, df = df_customer_unique)

end

@time p087()

  2.387073 seconds (2.83 M allocations: 155.500 MiB, 2.37% gc time, 93.69% compilation time)


(count = 21971,
 unique_count = 21941,
 diff = 30,
 df = 21941×12 DataFrame
   Row │ customer_name    postal_cd  customer_id     amount_sum  gender_cd  ge ⋯
       │ String31         String     String          Int64       String     St ⋯
───────┼────────────────────────────────────────────────────────────────────────
     1 │ おかやま あさみ  176-0002   CS019415000213        3272  1          女 ⋯
     2 │ おかやま そら    144-0046   CS001515000060         168  1          女
     3 │ おかやま ひかり  185-0011   CS002515000380        1623  1          女
     4 │ おかやま まひる  185-0012   CS002612000263           0  1          女
     5 │ おかやま まひる  285-0858   CS007713000059           0  1          女 ⋯
     6 │ おかやま まみ    241-0826   CS028314000010        2348  1          女
     7 │ おかやま めぐみ  214-0038   CS024114000006         628  1          女
     8 │ おかやま 一恵    242-0024   CS025413000064         316  1          女
     9 │ おかやま 京子    167-0051   CS039415000296           0  1          女 ⋯
    10 │ おかやま 倫子    182-0022   C

---
> P-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

In [95]:
function p088()
    p087res = p087()
    df = @chain df_customer begin
        leftjoin(
            p087res.df[!, [:customer_name, :postal_cd, :customer_id]],
            on=[:customer_name, :postal_cd],
            renamecols="" => "_unique"
        )
        rename(:customer_id => :customer_id_original, :customer_id_unique => :integrated_customer_id)
    end

    orig = length(unique(df.customer_id_original))
    new = length(unique(df.integrated_customer_id))
    (customer_id = orig, integrated_customer_id = new, diff = orig - new, df = df)
end

@time p088()

  1.936251 seconds (2.71 M allocations: 161.275 MiB, 2.77% gc time, 89.77% compilation time)


(customer_id = 21971,
 integrated_customer_id = 21941,
 diff = 30,
 df = 21971×12 DataFrame
   Row │ customer_id_original  customer_name  gender_cd  gender   birth_day    ⋯
       │ String                String31       String     String7  Date         ⋯
───────┼────────────────────────────────────────────────────────────────────────
     1 │ CS021313000114        大野 あや子    1          女性     1981-04-29   ⋯
     2 │ CS037613000071        六角 雅彦      9          不明     1952-04-01
     3 │ CS031415000172        宇多田 貴美子  1          女性     1976-10-04
     4 │ CS028811000001        堀井 かおり    1          女性     1933-03-27
     5 │ CS001215000145        田崎 美紀      1          女性     1995-03-29   ⋯
     6 │ CS020401000016        宮下 達士      0          男性     1974-09-15
     7 │ CS015414000103        奥野 陽子      1          女性     1977-08-09
     8 │ CS029403000008        釈 人志        0          男性     1973-08-17
     9 │ CS015804000004        松谷 米蔵      0          男性     1931-05-02   ⋯
    10 │ CS033513

---
> P-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [96]:
function p089()
    customers = @chain df_receipt begin
        groupby(:customer_id)
        @combine :amount_sum = sum(:amount)
    end

    df = innerjoin(df_customer, customers, on=:customer_id)
    train, test = partition(df, 0.8)
    println("train: $(nrow(train)/nrow(df)), test: $(nrow(test)/nrow(df))")

    (train = train, test = test)
end

@time p089()

train: 0.8000240789790513, test: 0.19997592102094872
  0.645822 seconds (1.47 M allocations: 81.876 MiB, 7.94% gc time, 96.79% compilation time)


(train = 6645×12 DataFrame
  Row │ customer_id     customer_name  gender_cd  gender   birth_day   age     ⋯
      │ String          String31       String     String7  Date        Int64   ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │ CS031415000172  宇多田 貴美子  1          女性     1976-10-04     42   ⋯
    2 │ CS001215000145  田崎 美紀      1          女性     1995-03-29     24
    3 │ CS015414000103  奥野 陽子      1          女性     1977-08-09     41
    4 │ CS033513000180  安斎 遥        1          女性     1962-07-11     56
    5 │ CS011215000048  芦田 沙耶      1          女性     1992-02-01     27   ⋯
    6 │ CS040412000191  川井 郁恵      1          女性     1977-01-05     42
    7 │ CS029415000023  梅田 里穂      1          女性     1976-01-17     43
    8 │ CS009315000023  皆川 文世      1          女性     1980-04-15     38
    9 │ CS035415000029  寺沢 真希      9          不明     1977-09-27     41   ⋯
   10 │ CS015315000033  福士 璃奈子    1          女性     1983-03-17     36
   11 │ CS

---
> P-090: レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

In [97]:
function select_train_test(a)
    selected = sample(a, 18, replace=false)
    (selected[1:12], selected[13:18])
end

function p090()
    df = @chain df_receipt begin
        @rtransform :yearmonth = string(:sales_ymd)[1:6]
        groupby(:yearmonth)
        @combine :amount_sum = sum(:amount)
    end
    ym_train, ym_test = select_train_test(df.yearmonth)
    train = subset(df, :yearmonth => ByRow(x -> x in ym_train))
    test = subset(df, :yearmonth => ByRow(x -> x in ym_test))
    (train = train, test = test)
end

@time [p090(), p090(), p090()]

  0.631099 seconds (1.57 M allocations: 98.048 MiB, 8.18% gc time, 82.82% compilation time)


3-element Vector{NamedTuple{(:train, :test), Tuple{DataFrame, DataFrame}}}:
 (train = 12×2 DataFrame
 Row │ yearmonth  amount_sum 
     │ String     Int64      
─────┼───────────────────────
   1 │ 201707         959205
   2 │ 201909        1105696
   3 │ 201910        1143062
   4 │ 201803         946588
   5 │ 201807        1058472
   6 │ 201908        1133614
   7 │ 201702         764413
   8 │ 201905        1111985
   9 │ 201907        1118371
  10 │ 201901        1064085
  11 │ 201709         902037
  12 │ 201801         944509, test = 6×2 DataFrame
 Row │ yearmonth  amount_sum 
     │ String     Int64      
─────┼───────────────────────
   1 │ 201701         902056
   2 │ 201703         962945
   3 │ 201711         932157
   4 │ 201712         939654
   5 │ 201904        1044210
   6 │ 201810        1069939)
 (train = 12×2 DataFrame
 Row │ yearmonth  amount_sum 
     │ String     Int64      
─────┼───────────────────────
   1 │ 201902         959538
   2 │ 201906        1089063
 

---
> P-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [98]:
function p091()
    df = @chain df_customer begin
        leftjoin(df_receipt[!, [:customer_id, :amount]], on=:customer_id)
        groupby(:customer_id)
        @combine :amount_sum = coalesce.(sum(:amount), 0)
        @rtransform :sales_flag = ifelse(:amount_sum > 0, 1, 0)
        innerjoin(df_customer, on=:customer_id)
    end

    df_sample, _ = undersample(df, df.sales_flag) |> getobs

    @chain df_sample begin
        groupby(:sales_flag)
        @combine $nrow
    end
end

@time p091()

  0.807378 seconds (1.01 M allocations: 80.673 MiB, 95.40% compilation time)


Row,sales_flag,nrow
,Int64,Int64
1,0,8306
2,1,8306


---
> P-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

In [99]:
function p092()
    df = @select df_customer $(Not(:gender))
    dfg = @distinct df_customer[!, [:gender_cd, :gender]] :gender_cd :gender
    (df_customer_std = df, df_gender_std = dfg)
end

@time p092()

  0.396777 seconds (638.75 k allocations: 35.521 MiB, 10.69% gc time, 99.13% compilation time)


(df_customer_std = 21971×10 DataFrame
   Row │ customer_id     customer_name  gender_cd  birth_day   age    postal_c ⋯
       │ String          String31       String     Date        Int64  String   ⋯
───────┼────────────────────────────────────────────────────────────────────────
     1 │ CS021313000114  大野 あや子    1          1981-04-29     37  259-1113 ⋯
     2 │ CS037613000071  六角 雅彦      9          1952-04-01     66  136-0076
     3 │ CS031415000172  宇多田 貴美子  1          1976-10-04     42  151-0053
     4 │ CS028811000001  堀井 かおり    1          1933-03-27     86  245-0016
     5 │ CS001215000145  田崎 美紀      1          1995-03-29     24  144-0055 ⋯
     6 │ CS020401000016  宮下 達士      0          1974-09-15     44  174-0065
     7 │ CS015414000103  奥野 陽子      1          1977-08-09     41  136-0073
     8 │ CS029403000008  釈 人志        0          1973-08-17     45  279-0003
     9 │ CS015804000004  松谷 米蔵      0          1931-05-02     87  136-0073 ⋯
    10 │ CS033513000180  安斎 遥        1   

---
> P-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

In [100]:
function p093()
    innerjoin(
        df_product,
        df_category,
        on=[:category_major_cd , :category_medium_cd, :category_small_cd]
    )
end

@time p093()

  0.783348 seconds (1.38 M allocations: 66.430 MiB, 99.82% compilation time)


Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
,String,String,String,String,Int64?,Int64?,String31,String,String
1,P040101001,04,0401,040101,198,149,惣菜,御飯類,弁当類
2,P040101002,04,0401,040101,218,164,惣菜,御飯類,弁当類
3,P040101003,04,0401,040101,230,173,惣菜,御飯類,弁当類
4,P040101004,04,0401,040101,248,186,惣菜,御飯類,弁当類
5,P040101005,04,0401,040101,268,201,惣菜,御飯類,弁当類
6,P040101006,04,0401,040101,298,224,惣菜,御飯類,弁当類
7,P040101007,04,0401,040101,338,254,惣菜,御飯類,弁当類
8,P040101008,04,0401,040101,420,315,惣菜,御飯類,弁当類
9,P040101009,04,0401,040101,498,374,惣菜,御飯類,弁当類


---
> P-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [101]:
@time \
CSV.write("data/094.csv", p093())

  0.000001 seconds


"data/094.csv"

---
> P-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [102]:
# できなそう

---
> P-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [103]:
@time \
CSV.write("data/096.csv", p093(), header=false)

  0.000001 seconds


"data/096.csv"

---
> P-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

In [104]:
@time \
CSV.File("data/094.csv", types=Dict(r"_cd$" => String)) |>
    DataFrame |>
    df -> first(df, 3)

  0.000001 seconds


Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
,String,String,String,String,Int64?,Int64?,String31,String,String
1,P040101001,04,0401,040101,198,149,惣菜,御飯類,弁当類
2,P040101002,04,0401,040101,218,164,惣菜,御飯類,弁当類
3,P040101003,04,0401,040101,230,173,惣菜,御飯類,弁当類


---
> P-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

In [105]:
@time \
CSV.File(
    "data/096.csv",
    header=[
        :product_cd, :category_major_cd, :category_medium_cd, :category_small_cd,
        :unit_price, :unit_cost, :category_major_name, :category_medium_name, :category_small_name
    ],
    types=Dict(r"_cd$" => String)
) |> DataFrame |> df -> first(df, 3)

  0.000004 seconds


Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
,String,String,String,String,Int64?,Int64?,String31,String,String
1,P040101001,04,0401,040101,198,149,惣菜,御飯類,弁当類
2,P040101002,04,0401,040101,218,164,惣菜,御飯類,弁当類
3,P040101003,04,0401,040101,230,173,惣菜,御飯類,弁当類


---
> P-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [106]:
@time \
CSV.write("data/099.csv", p093(), delim = '\t')

  0.000002 seconds


"data/099.csv"

---
> P-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

In [107]:
@time \
CSV.File("data/099.csv", delim='\t', types=Dict(r"_cd$" => String)) |>
    DataFrame |>
    df -> first(df, 3)

  0.000001 seconds


Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
,String,String,String,String,Int64?,Int64?,String31,String,String
1,P040101001,04,0401,040101,198,149,惣菜,御飯類,弁当類
2,P040101002,04,0401,040101,218,164,惣菜,御飯類,弁当類
3,P040101003,04,0401,040101,230,173,惣菜,御飯類,弁当類


# これで１００本終わりです。おつかれさまでした！